In [1]:
from operator import *

from nltk.parse import stanford, corenlp
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [2]:
input_dir = '/data/yu_gai/cfq'
output_dir = '/work/yu_gai/cfq/data/cfq'

df = sqlCtx.read.parquet(f'{input_dir}/dataset.parquet').sort('index').persist()
df.columns

['complexityMeasures',
 'expectedResponse',
 'expectedResponseWithMids',
 'index',
 'question',
 'questionPatternModEntities',
 'questionTemplate',
 'questionWithBrackets',
 'questionWithMids',
 'ruleIds',
 'ruleTree',
 'sparql',
 'sparqlPattern',
 'sparqlPatternModEntities']

In [4]:
def replace(q):
    for s in [
        'art director',
        'country of nationality',
        'costume designer',
        'executive producer',
        'executive produce',
        'executive produced',
        'film director',
        'film distributor',
        'film editor',
        'film producer',
        'production company',
    ]:
        q = q.replace(s, s.replace(' ', ''))
    return q

df = df.withColumn('questionPatternModEntities', udf(replace, StringType())('questionPatternModEntities')).persist()
df.rdd.map(lambda r: len(r['questionPatternModEntities'].split(' ')) == len(r['questionTemplate'].split(' '))).reduce(and_)

True

In [8]:
trees = df.rdd.map(lambda r: r['questionTemplate'].replace('[', '').replace(']', '').replace('VP_SIMPLE', 'do').replace('ADJECTIVE_SIMPLE', 'good')).distinct().map(lambda r: next(parser.raw_parse(r)))

In [9]:
for tree in trees.take(10):
    tree.pretty_print()

                       ROOT                                                              
                        |                                                                 
                        S                                                                
                        |                                                                 
                        VP                                                               
  ______________________|___________________________                                      
 |                                                 SBAR                                  
 |                                                  |                                     
 |                                                  S                                    
 |                       ___________________________|____________                         
 |                      NP                                       VP                      
 |   

In [23]:
def rephrase(q):
    for k, v in {
        '[ADJECTIVE_SIMPLE]' : 'big',
        '[entity]' : 'something',
        '[NP_SIMPLE]' : 'something',
        '[ROLE_SIMPLE]' : 'something',
        '[VP_SIMPLE]' : 'do',
    }.items():
        q = q.replace(k, v)
    return q

In [21]:
q = "What [NP_SIMPLE] that [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity] was a [NP_SIMPLE] 's [ROLE_SIMPLE]"
t = next(parser.raw_parse(rephrase(q)))
t.pretty_print()

                    ROOT                                                                        
                     |                                                                           
                    FRAG                                                                        
                     |                                                                           
                    SBAR                                                                        
  ___________________|______________________                                                     
 |                                          S                                                   
 |               ___________________________|____________________________                        
 |              NP                                                       |                      
 |        ______|________________                                        |                       
 |       |               

In [147]:
parser = corenlp.CoreNLPParser()
for tree in parser.raw_parse(df.rdd.map(lambda r: r['question']).take(1)[0]):
    tree.pretty_print()

                           ROOT                               
                            |                                  
                            S                                 
                            |                                  
                            VP                                
                 ___________|_____________________             
                VP                    |           |           
  ______________|____                 |           |            
 |                   NP               |           VP          
 |           ________|___________     |      _____|___         
 |          NP       |      |    |    |     |         NP      
 |     _____|___     |      |    |    |     |      ___|____    
VBD  NNP       POS   JJ     NN   NN   CC    VB    NN       NN 
 |    |         |    |      |    |    |     |     |        |   
Did Jackie      's female actor edit and produce Rad     Plaid



In [148]:
df.rdd.map(lambda r: len(list(parser.raw_parse(r['question'])))).distinct().collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 64 in stage 20.0 failed 1 times, most recent failure: Lost task 64.0 in stage 20.0 (TID 1101, havoc.millennium.berkeley.edu, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/data/yu_gai/anaconda3/envs/ee227/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/data/yu_gai/anaconda3/envs/ee227/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 595, in process
    out_iter = func(split_index, iterator)
  File "/data/yu_gai/anaconda3/envs/cfq-yu/lib/python3.8/site-packages/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/data/yu_gai/anaconda3/envs/cfq-yu/lib/python3.8/site-packages/pyspark/rdd.py", line 2

In [119]:
parser = corenlp.CoreNLPParser()
q = "do and do"
t = next(parser.raw_parse(rephrase(q)))
t.pretty_print()

    ROOT    
     |       
     S      
     |       
     VP     
  ___|____   
 VB  CC   VB
 |   |    |  
 do and   do



In [143]:
grammar = nltk.CFG.fromstring("""
    S -> WHNP VP | 'Did' NP VP
    CC -> 'and'
    DT -> 'a' | 'an'
    JJ -> 'ADJECTIVE_SIMPLE'
    N -> 'NP_SIMPLE' | 'ROLE_SIMPLE'
    NP -> 'entity' | DT N | DT JJ N | NP POS N
    V -> 'VP_SIMPLE'
    VP -> 'was' V PP | VP CC VP
    WHNP -> 'Who' | 'What' N
    POS -> "'s"
    P -> 'by'
    PP -> P NP
""")
parser = nltk.ChartParser(grammar)
# parser = nltk.ShiftReduceParser(grammar)

sent = "What [NP_SIMPLE] was [VP_SIMPLE] by [entity] and was [VP_SIMPLE] by a [ADJECTIVE_SIMPLE] [NP_SIMPLE] 's [ROLE_SIMPLE]".replace('[', '').replace(']', '')
trees = list(parser.parse(sent.split(' ')))
print(len(trees))
trees[0].pretty_print()

In [100]:
sent

"What NP_SIMPLE was VP_SIMPLE by entity and was VP_SIMPLE by a ADJECTIVE_SIMPLE NP_SIMPLE 's ROLE_SIMPLE"

In [52]:
sent.split(' ')

['a', 'NP_SIMPLE', "'s"]

In [16]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))

In [27]:
import nltk

In [39]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")
parser = nltk.ChartParser(grammar)

In [40]:
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
tree = next(parser.parse(sent))
tree.pretty_print()

     S                                       
  ___|______________                          
 |                  VP                       
 |         _________|__________               
 |        VP                   PP            
 |    ____|___              ___|___           
 |   |        NP           |       NP        
 |   |     ___|_____       |    ___|_____     
 NP  V   Det        N      P  Det        N   
 |   |    |         |      |   |         |    
 I  shot  an     elephant  in  my     pajamas



In [41]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'an' | 'my'
    N -> 'NP_SIMPLE' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")
parser = nltk.ChartParser(grammar)